# Imports

In [2]:
import sys
import pathlib
import joblib
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import itertools

from functools import partial
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

if pathlib.Path().parent.resolve().absolute().as_posix() not in sys.path:
    sys.path.append(pathlib.Path().parent.resolve().absolute().as_posix())

from pilot import Pilot, ensemble, Tree

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning

OUTPUTPATH = pathlib.Path().absolute() / 'Output'

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

%load_ext autoreload
%autoreload 2

# Functions

In [3]:
def highlight(data, kind='max', color='black', background_color='lightgreen'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'color: {}; background-color: {}'.format(color, background_color)
    #remove % and cast to float
    data = data.replace('%','', regex=True).astype(float)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        if kind == 'max':
            highlight = data == data.max()
        elif kind == 'min':
            highlight = data == data.min()
        return [attr if v else '' for v in highlight]
    else:  # from .apply(axis=None)
        if kind == 'max':
            highlight = data == data.max().max()
        elif kind == 'min':
            highlight = data == data.min().min()
        return pd.DataFrame(np.where(highlight, attr, ''),
                            index=data.index, columns=data.columns)

# Load data

In [4]:
DATAPATH = pathlib.Path().absolute() / 'Data'
datasets = {
    'abalone': {'categorical': [0]}, # sex
    'airfoil': {'categorical': [-1]},
    'Bias_correction_ucl': {'categorical': [0]}, # station 
    'bodyfat_preprocessed': {'categorical': [-1]},
    'boston': {'categorical': [3]}, # Charles river dummy
    'communities': {'categorical': [-1]}, # 119 = LemasGangUnitDeploy
    'concrete': {'categorical': [-1]},
    'diabetes': {'categorical': [-1]},
    'electricity': {'categorical': [-1]},
    'energy': {'categorical': [5]}, # Orientation (X6)
    'ga_preprocessed': {'categorical': [6]}, # Research
    'housing': {'categorical': [-1]},
    'ozone_preprocessed': {'categorical': [10, 11]},
    'residential': {'categorical': [-1]},
    'ribo_preprocessed': {'categorical': [-1]},
    'SeoulBikeData': {'categorical': [-1]}, # 10 = seasons
    'skills': {'categorical': [-1]},
    'superconductor': {'categorical': [-1]},
    'Walmart_preprocessed': {'categorical': [0]},
    'wine': {'categorical': [-1]}
}

In [5]:
for d in datasets:
    df = pd.read_csv(DATAPATH / f'{d}.csv')
    X = df.drop(columns='target').values
    y = df['target'].values
    datasets[d]['data'] = X, y


# Grid Search

## Run grid

In [70]:
OUTPUTPATH = pathlib.Path().absolute() / 'Output'

filename = OUTPUTPATH / 'rf_gridsearch_final.pkl'

if filename.exists():
    results = joblib.load(filename)
else:
    results = {}

In [74]:
datasets.keys(), results.keys()

(dict_keys(['abalone', 'airfoil', 'Bias_correction_ucl', 'bodyfat_preprocessed', 'boston', 'communities', 'concrete', 'diabetes', 'electricity', 'energy', 'ga_preprocessed', 'housing', 'ozone_preprocessed', 'residential', 'ribo_preprocessed', 'SeoulBikeData', 'skills', 'superconductor', 'Walmart_preprocessed', 'wine']),
 dict_keys(['abalone', 'airfoil', 'bodyfat_preprocessed', 'boston', 'concrete', 'diabetes', 'electricity', 'energy', 'ga_preprocessed', 'ozone_preprocessed', 'residential', 'SeoulBikeData', 'skills', 'Walmart_preprocessed', 'wine']))

In [72]:
datasets_to_ignore = ['superconductor', 'Bias_correction_ucl', 'communities', 'housing', 'ribo_preprocessed']

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
for i, (d, data) in enumerate(datasets.items()):
    print(i, d)
    if (d in datasets_to_ignore) or (d in results.keys()):
        continue
    pilot = GridSearchCV(
        estimator=ensemble.RandomForestPilot(),
        param_grid={
            'n_estimators': [100],
            'max_depth': [10],
            'truncation_factor': [1, 1.5],
            'min_sample_split': [2],
            'min_sample_leaf':  [1],
            'n_features': [0.3, 0.7, 1.0],
            'rel_tolerance': [0.01],
            'df_settings': [{'con': i, 'lin': j} for i, j in itertools.product([1, 5], [2, 5])],
            'min_unique_values_regression': [2, 5]
            
        }, 
        scoring='neg_mean_squared_error',
        cv=cv,
        n_jobs=-1,
        verbose=10
    )
    tree = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'n_estimators': [100],
            'max_depth': [10],
            'min_samples_split': [2],
            'min_samples_leaf':  [1],
            'max_features': [0.3, 0.7, 1.0]
        },
        scoring='neg_mean_squared_error', 
        cv=cv,
        n_jobs=-1
    )

    X, y = data['data']
    
    pilot.fit(X, y, categorical_idx=np.array(data['categorical']))
    tree.fit(X, y)

    
    print(d, round(pilot.best_score_, 2), round(tree.best_score_, 2))
    
    results[d] = {'pilot': pilot.cv_results_, 'rf': tree.cv_results_}
    joblib.dump(results, filename)

## Inspect results

In [7]:
OUTPUTPATH = pathlib.Path().absolute() / 'Output'
results = joblib.load( OUTPUTPATH / 'rf_gridsearch_final.pkl')

In [75]:
normalized_results = pd.DataFrame([{
    'dataset': d, 
    'pilot_mse': -np.nanmax(r['pilot']['mean_test_score']), 
    'rf_mse': -np.nanmax(r['rf']['mean_test_score'])
    } for d, r in results.items()]).set_index('dataset').apply(lambda row: row / row.min(), axis=1)

In [76]:
normalized_results.style.apply(partial(highlight, kind='min'), axis=1).format('{:.3f}')

,pilot_mse,rf_mse
dataset,,
abalone,1.000,1.010
airfoil,1.000,1.673
bodyfat_preprocessed,1.000,1.265
boston,1.053,1.000
concrete,1.000,1.159
diabetes,1.000,1.076
electricity,1.000,1.385
energy,1.000,1.583
ga_preprocessed,1.000,1.066


In [77]:
df = pd.concat([
    pd.concat([
        pd.DataFrame(results[d]['pilot']).assign(dataset=d).reset_index(),
        pd.json_normalize(pd.DataFrame(results[d]['pilot'])['param_df_settings']).rename(columns=lambda c: 'param_df_' + c)
    ], axis=1).assign(best_rf=np.nanmax(results[d]['rf']['mean_test_score'])) for d in results
]).groupby(
    ['dataset'] + [c for c in results['abalone']['pilot'].keys() if c.startswith('param_') and c != 'param_df_settings'] + ['param_df_con', 'param_df_lin']
)[['mean_test_score', 'best_rf']].mean() * -1

df['mse_ratio'] = df['mean_test_score'] / df['best_rf']
# remove param combo's that were not tested for all datasets
df = df['mse_ratio'].unstack(level=0).loc[lambda df: (df.index.get_level_values('param_max_depth') == 10) & ~(df.index.get_level_values('param_df_con').astype(str) + df.index.get_level_values('param_df_lin').astype(str)).isin(['22', '25'])]

df['average'] = df.mean(axis=1)
df['count_better'] = (df.drop(columns='average') < 1).sum(axis=1)

In [78]:
df.sort_values(['count_better', 'average'], ascending=[False, True]).T

param_max_depth                            10                        \
param_min_sample_leaf                       1                         
param_min_sample_split                      2                         
param_min_unique_values_regression          2                         
param_n_estimators                        100                         
param_n_features                          1.0        0.7              
param_rel_tolerance                      0.01       0.01              
param_truncation_factor                   1.5        1.0        1.5   
param_df_con                                5          5          5   
param_df_lin                                5          5          5   
dataset                                                               
SeoulBikeData                        0.786038   0.805530   0.794413   
Walmart_preprocessed                 0.894787   0.865841   0.865535   
abalone                              0.994255   0.992397   0.989938   
airfoil                              0.600123   0.666019   0.669633   
bodyfat_preprocessed                 0.918760   0.851210   0.904338   
boston                               1.217297   1.193807   1.139658   
concrete                             0.905650   0.862629   0.870268   
diabetes                             0.995255   0.998078   0.990657   
electricity                          0.736316   0.749665   0.768069   
energy                               0.631806   0.760404   0.745010   
ga_preprocessed                      1.056460   1.014574   1.018995   
ozone_preprocessed                   1.109852   1.044701   1.059043   
residential                          0.630687   0.746467   0.750127   
skills                               0.985842   0.982436   0.978257   
wine                                 0.971915   0.968405   0.962260   
average                              0.895670   0.900144   0.900413   
count_better                        12.000000  12.000000  12.000000   

param_max_depth                                                      \
param_min_sample_leaf                                                 
param_min_sample_split                                                
param_min_unique_values_regression                     5              
param_n_estimators                                   100              
param_n_features                          1.0        0.7              
param_rel_tolerance                      0.01       0.01              
param_truncation_factor                   1.0        1.0        1.5   
param_df_con                                5          5          5   
param_df_lin                                5          5          5   
dataset                                                               
SeoulBikeData                        0.779449   0.938859   0.941188   
Walmart_preprocessed                 0.866862   0.842733   0.840777   
abalone                              1.000906   0.991328   0.995651   
airfoil                              0.597814   0.688618   0.681741   
bodyfat_preprocessed                 0.881920   0.908027   0.883396   
boston                               1.249513   1.129794   1.119003   
concrete                             0.909576   0.879964   0.883046   
diabetes                             0.977797   0.963075   0.982103   
electricity                          0.721931   0.753269   0.758483   
energy                               0.669385   1.112337   1.102683   
ga_preprocessed                      1.074789   1.002736   1.022775   
ozone_preprocessed                   1.072088   1.035772   1.046210   
residential                          0.616557   0.727100   0.721729   
skills                               0.988146   0.975511   0.981056   
wine                                 0.982302   0.962777   0.966966   
average                              0.892602   0.927460   0.928454   
count_better                        11.000000  11.000000  11.000000   

param_max_depth       

In [82]:
results['airfoil']['pilot'].keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_df_settings', 'param_max_depth', 'param_min_sample_leaf', 'param_min_sample_split', 'param_min_unique_values_regression', 'param_n_estimators', 'param_n_features', 'param_rel_tolerance', 'param_truncation_factor', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [83]:
df = pd.concat([
    pd.concat([
        pd.DataFrame(results[d]['pilot']).assign(dataset=d).reset_index(),
        pd.json_normalize(pd.DataFrame(results[d]['pilot'])['param_df_settings']).rename(columns=lambda c: 'param_df_' + c)
    ], axis=1) for d in results
]).groupby(
    ['dataset'] + [c for c in results['abalone']['pilot'].keys() if c.startswith('param_') and c != 'param_df_settings'] + ['param_df_con', 'param_df_lin']
)['mean_fit_time'].mean().unstack(level=0).loc[lambda df: (df.index.get_level_values('param_max_depth') == 10) & ~(df.index.get_level_values('param_df_con').astype(str) + df.index.get_level_values('param_df_lin').astype(str)).isin(['22', '25'])]

df['average'] = df.mean(axis=1)


In [86]:
df.sort_values('average', ascending=False).T

param_max_depth                              10                            \
param_min_sample_leaf                         1                             
param_min_sample_split                        2                             
param_min_unique_values_regression            2                         5   
param_n_estimators                          100                       100   
param_n_features                            1.0                       1.0   
param_rel_tolerance                        0.01                      0.01   
param_truncation_factor                     1.5          1.0          1.0   
param_df_con                                  5            5            5   
param_df_lin                                  2            2            2   
dataset                                                                     
SeoulBikeData                        591.946307   600.689023   468.056606   
Walmart_preprocessed                 274.096758   273.585674   250.491879   
abalone                               81.058273    83.525842    89.024745   
airfoil                               33.653573    33.991733    14.096375   
bodyfat_preprocessed                  21.863316    21.289592    20.712917   
boston                                44.004106    43.222365    53.011499   
concrete                              81.303153    77.932901    44.345036   
diabetes                              11.346950    11.440345    10.848298   
electricity                         2217.543054  2112.142107  2000.759984   
energy                                 7.784366     7.476793    20.655138   
ga_preprocessed                        3.454825     3.434207     3.190374   
ozone_preprocessed                     4.871748     4.838941     8.824503   
residential                          156.033993   153.756687   174.221375   
skills                               336.469408   329.008261   337.579113   
wine                                  99.393104    93.269452   101.427546   
average                              264.321529   256.640261   239.816359   

param_max_depth                                                            \
param_min_sample_leaf                                                       
param_min_sample_split                                                      
param_min_unique_values_regression            2            5            2   
param_n_estimators                          100          100          100   
param_n_features                            1.0          1.0          1.0   
param_rel_tolerance                        0.01         0.01         0.01   
param_truncation_factor                     1.5          1.5          1.0   
param_df_con                                  1            5            1   
param_df_lin                                  2            2            2   
dataset                                                                     
SeoulBikeData                        398.750226   500.705917   364.488468   
Walmart_preprocessed                 198.881071   246.920173   193.200664   
abalone                              104.315097    86.032984    93.768481   
airfoil                               13.567928    14.095440    12.974417   
bodyfat_preprocessed                   7.541303    20.835050     7.504929   
boston                                26.966270    52.051166    26.499841   
concrete                              29.181865    44.161832    28.889335   
diabetes                               8.516861    10.142951     8.539936   
electricity                         2253.560109  1903.954610  2251.678108   
energy                                 3.465992    21.138024     3.130007   
ga_preprocessed                        2.265439     3.412520     1.994752   
ozone_preprocessed                     2.472146     9.250473     2.496621   
residential                          124.667424   173.199617   114.949762   
skills                               289.008151   323.338525   281.46328

In [98]:
df.stack().reset_index().groupby(['dataset', 'param_min_unique_values_regression'])[0].mean().unstack()

param_min_unique_values_regression,2,5
dataset,,
SeoulBikeData,268.402972,235.964836
Walmart_preprocessed,147.363678,136.180230
abalone,59.721499,60.384274
airfoil,13.060156,10.218217
average,136.917102,128.725745
bodyfat_preprocessed,9.713924,9.673420
boston,19.422069,20.638361
concrete,25.321328,19.848678
diabetes,7.861885,7.638603


# Grid search difficult datasets

## Run grid

In [8]:
OUTPUTPATH = pathlib.Path().absolute() / 'Output'

filename = OUTPUTPATH / 'rf_gridsearch_final_other.pkl'

if filename.exists():
    results = joblib.load(filename)
else:
    results = {}

In [9]:
datasets.keys(), results.keys()

(dict_keys(['abalone', 'airfoil', 'Bias_correction_ucl', 'bodyfat_preprocessed', 'boston', 'communities', 'concrete', 'diabetes', 'electricity', 'energy', 'ga_preprocessed', 'housing', 'ozone_preprocessed', 'residential', 'ribo_preprocessed', 'SeoulBikeData', 'skills', 'superconductor', 'Walmart_preprocessed', 'wine']),
 dict_keys([]))

In [10]:
datasets_to_handle = ['Bias_correction_ucl', 'communities', 'housing', 'ribo_preprocessed']

In [ ]:
cv = KFold(n_splits=4, shuffle=True, random_state=42)
for i, (d, data) in enumerate(datasets.items()):
    print(i, d)
    if (d not in datasets_to_handle) or (d in results.keys()):
        continue
    pilot = GridSearchCV(
        estimator=ensemble.RandomForestPilot(),
        param_grid={
            'n_estimators': [100],
            'max_depth': [10],
            'truncation_factor': [1, 1.5],
            'min_sample_split': [2],
            'min_sample_leaf':  [1],
            'n_features': [0.3, 0.7, 1.0],
            'rel_tolerance': [0.01],
            'df_settings': [{'con': 1, 'lin': 2}, {'con': 5, 'lin': 5}, {'con': 5, 'lin': 2}],
            'min_unique_values_regression': [4, 5]
            
        }, 
        scoring='neg_mean_squared_error',
        cv=cv,
        n_jobs=-1,
        verbose=10
    )
    tree = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'n_estimators': [100],
            'max_depth': [10],
            'min_samples_split': [2],
            'min_samples_leaf':  [1],
            'max_features': [0.3, 0.7, 1.0]
        },
        scoring='neg_mean_squared_error', 
        cv=cv,
        n_jobs=-1
    )

    X, y = data['data']
    
    pilot.fit(X, y, categorical_idx=np.array(data['categorical']))
    tree.fit(X, y)

    
    print(d, round(pilot.best_score_, 2), round(tree.best_score_, 2))
    
    results[d] = {'pilot': pilot.cv_results_, 'rf': tree.cv_results_}
    joblib.dump(results, filename)

## Inspect results

In [ ]:
OUTPUTPATH = pathlib.Path().absolute() / 'Output'
results = joblib.load( OUTPUTPATH / 'rf_gridsearch_final_other.pkl')

In [13]:
normalized_results = pd.DataFrame([{
    'dataset': d, 
    'pilot_mse': -np.nanmax(r['pilot']['mean_test_score']), 
    'rf_mse': -np.nanmax(r['rf']['mean_test_score'])
    } for d, r in results.items()]).set_index('dataset').apply(lambda row: row / row.min(), axis=1)

In [14]:
normalized_results.style.apply(partial(highlight, kind='min'), axis=1).format('{:.3f}')

,pilot_mse,rf_mse
dataset,,
Bias_correction_ucl,1.000,1.180
communities,1.000,1.053


In [16]:
df = pd.concat([
    pd.concat([
        pd.DataFrame(results[d]['pilot']).assign(dataset=d).reset_index(),
        pd.json_normalize(pd.DataFrame(results[d]['pilot'])['param_df_settings']).rename(columns=lambda c: 'param_df_' + c)
    ], axis=1).assign(best_rf=np.nanmax(results[d]['rf']['mean_test_score'])) for d in results
]).groupby(
    ['dataset'] + [c for c in list(results.values())[0]['pilot'].keys() if c.startswith('param_') and c != 'param_df_settings'] + ['param_df_con', 'param_df_lin']
)[['mean_test_score', 'best_rf']].mean() * -1

df['mse_ratio'] = df['mean_test_score'] / df['best_rf']
# remove param combo's that were not tested for all datasets
df = df['mse_ratio'].unstack(level=0).loc[lambda df: (df.index.get_level_values('param_max_depth') == 10) & ~(df.index.get_level_values('param_df_con').astype(str) + df.index.get_level_values('param_df_lin').astype(str)).isin(['22', '25'])]

df['average'] = df.mean(axis=1)
df['count_better'] = (df.drop(columns='average') < 1).sum(axis=1)

In [17]:
df.sort_values(['count_better', 'average'], ascending=[False, True]).T

param_max_depth                           10                                \
param_min_sample_leaf                      1                                 
param_min_sample_split                     2                                 
param_min_unique_values_regression         4                   5         4   
param_n_estimators                       100                 100       100   
param_n_features                         0.3                 0.3       0.7   
param_rel_tolerance                     0.01                0.01      0.01   
param_truncation_factor                  1.0       1.5       1.0       1.5   
param_df_con                               5         5         5         5   
param_df_lin                               2         2         2         2   
dataset                                                                      
Bias_correction_ucl                 0.847544  0.865949  0.862982  0.881355   
communities                         0.956044  0.951650  0.960536  0.949789   
average                             0.901794  0.908800  0.911759  0.915572   
count_better                        2.000000  2.000000  2.000000  2.000000   

param_max_depth                                                             \
param_min_sample_leaf                                                        
param_min_sample_split                                                       
param_min_unique_values_regression         5                   4         5   
param_n_estimators                       100                 100       100   
param_n_features                         0.3       0.7       0.7       0.7   
param_rel_tolerance                     0.01      0.01      0.01      0.01   
param_truncation_factor                  1.5       1.5       1.0       1.0   
param_df_con                               5         5         5         5   
param_df_lin                               2         2         2         2   
dataset                                                                      
Bias_correction_ucl                 0.878173  0.886513  0.887175  0.885362   
communities                         0.955659  0.957671  0.957504  0.959755   
average                             0.916916  0.922092  0.922340  0.922558   
count_better                        2.000000  2.000000  2.000000  2.000000   

param_max_depth                                         ...            \
param_min_sample_leaf                                   ...             
param_min_sample_split                                  ...             
param_min_unique_values_regression                   4  ...             
param_n_estimators                                 100  ...             
param_n_features                                   0.7  ...       1.0   
param_rel_tolerance                               0.01  ...      0.01   
param_truncation_factor                  1.5       1.5  ...       1.0   
param_df_con                               1         1  ...         5   
param_df_lin                               2         2  ...         2   
dataset                                                 ...             
Bias_correction_ucl                 0.924000  0.920827  ...  0.976669   
communities                         0.951283  0.962140  ...  0.962599   
average                             0.937641  0.941484  ...  0.969634   
count_better                        2.000000  2.000000  ...  2.000000   

param_max_depth                                                             \
param_min_sample_leaf                                                        
param_min_sample_split                                                       
param_min_unique_values_regression         5         4         5         4   
param_n_estimators                       100       100       100       100   
param_n_features                         1.0       1.0       1.0       1.0   
param_rel_tolerance                     0.01      0.01      0.01      0.01   
param_truncation

# Check old results

In [60]:
results_old = joblib.load( OUTPUTPATH / 'rf_gridsearch_incl_df_settings.pkl')

In [81]:
ds = 'ozone_preprocessed'
best_idx = results_old[ds]['pilot']['mean_test_score'].argmax()
results_old[ds]['pilot']['params'][best_idx]

{'df_settings': {'con': 5, 'lin': 5},
 'max_depth': 10,
 'min_sample_leaf': 1,
 'min_sample_split': 2,
 'n_estimators': 20,
 'n_features': 0.5,
 'rel_tolerance': 0.01,
 'truncation_factor': 1.5}